## Вариант 1

1. Используйте тексты из предыдущего задания и обучите на них модель векторного представления слов (опционально можно приводить слова к нормальной форме и удалить стоп-слова). Можно использовать `gensim`.

2. Разделите коллекцию текстов на обучающее и тестовое множество. С помощью обученной модели векторного представления отобразите каждый документ в вектор, усреднив все вектора для слов документа. 

3. Используйте какой-либо алгоритм классификации (например `SVM`) для классификации текстов. Для обучения используйте тестовое множество, для анализа результатов - тестовое.

4. Простое усреднение векторов слов обычно не дает хорошего отображения документа в вектор. Придумайте альтернативный способ. Протестируйте его, повторно обучив алгоритм классификации на тех же данных. 

## 1) Обучим w2v и удалим стоп слова

In [12]:
import gzip
import numpy as np
import os
import random
from dataclasses import dataclass
from typing import Iterator, List
import nltk
import string
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [13]:
#считаем данные
WORDVEC_LEN = 300
SEED = 0

@dataclass
class Text:
    label: str
    title: str
    text: str

def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

In [14]:
texts = list(read_texts("data/news.txt.gz"))
print(f"Количество {len(texts)}")

Количество 10000


In [15]:
# загрузим стоп-слова
nltk.download('stopwords')
stopwords_ru = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bayut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# разобъём текст по словам
def tokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = nltk.WordPunctTokenizer().tokenize(text)
    return words

#нормализация текста, удаление знаков препинаний и стоп слов
def normalize_text(words: str) -> str:
    filtered_words = [w for w in words if all(c not in string.punctuation for c in w)]
    words = [word for word in filtered_words if word not in stopwords_ru]
    return words

In [20]:
sample_text = texts[0].text
tokens = tokenize_text(sample_text)
normalized = normalize_text(tokens)

print(f"Исходный текст: \n{sample_text}")
print(f"После токенизации и нормализации: \n{' '.join(normalized)}")

Исходный текст: 
Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы от швейцарского бренда Rolex. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру» в среду, 8 мая. Rolex Yacht-Master 40 Фото: пресс-служба Mercury Соревнования будут проходить с 10 по 18 июня. Первый этап: ночной переход из Сан-Ремо в Сен-Тропе 10-11 июня (дистанция 50 морских миль — около 90 километров). Второй этап: серия прибрежных гонок в бухте Сен-Тропе с 11 по 14 июня. Финальный этап пройдет с 15 по 18 июня: оффшорная гонка по маршруту Сен-Тропе — Генуя (243 морских мили — 450 километров). Маршрут проходит через скалистый остров Джиралья к северу от Корсики и завершается в Генуе.Регата, с 1997 года проходящая при поддержке Rolex, считается одной из самых значительных яхтенных гонок в Средиземноморье. В этом году в ней ожидается участие трех российски

In [21]:
# нормализация данных для word2vec
sentences = [normalize_text(tokenize_text(text.text)) for text in texts]

# обучение модели
w2v = Word2Vec(sentences, vector_size=WORDVEC_LEN, sg=1, seed=SEED, workers=1, min_count=50, window=50)

print("Выполнено!")

Выполнено!


In [23]:
print("Примеры похожих слов:")
print("Для слова 'новости':", w2v.wv.most_similar("новости"))
print("Для слова 'спорт':", w2v.wv.most_similar("спорт"))

Примеры похожих слов:
Для слова 'новости': [('риа', 0.9235125780105591), ('интерфакс', 0.5452293157577515), ('тасс', 0.5287728905677795), ('сергей', 0.4358040392398834), ('сегодня', 0.4345269799232483), ('владимир', 0.4302092492580414), ('дмитрий', 0.42994630336761475), ('рф', 0.4222298860549927), ('алексея', 0.417143851518631), ('агентства', 0.3930620551109314)]
Для слова 'спорт': [('р', 0.6705067753791809), ('экспресс', 0.6555298566818237), ('советский', 0.62300705909729), ('сборной', 0.5687049031257629), ('виталий', 0.5682256817817688), ('спорта', 0.5678483247756958), ('тренер', 0.5645512342453003), ('чемпионат', 0.562313973903656), ('чемпионате', 0.5578190088272095), ('мутко', 0.5553413033485413)]


## 2) Разделите коллекцию текстов на обучающее и тестовое множество. С помощью обученной модели векторного представления отобразите каждый документ в вектор, усреднив все вектора для слов документа. 

In [26]:
X = [text.text for text in texts]
y = [text.label for text in texts]

raw_train, raw_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)

In [27]:
print(f"Размер обучающей выборки: {len(raw_train)}")
print(f"Размер тестовой выборки: {len(raw_test)}")

Размер обучающей выборки: 8000
Размер тестовой выборки: 2000


In [28]:
def get_doc_emb_avg(doc):
    res = np.zeros(WORDVEC_LEN)
    tokens = normalize_text(tokenize_text(doc))
    valid_tokens = [word for word in tokens if word in w2v.wv.key_to_index]
    if not valid_tokens:
        return res
    for word in valid_tokens:
        res += w2v.wv.get_vector(word)
    return res / len(valid_tokens)

X_train = [get_doc_emb_avg(doc) for doc in raw_train]
X_test = [get_doc_emb_avg(doc) for doc in raw_test]

#нормализация
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 3. Обучим Naive Bayes классификатор:

In [29]:
#переведём метки в числовые обозначения
topics = {'science':0, 'style':1, 'culture':2, 'life':3, 'economics':4, 
          'business':5, 'travel':6, 'forces':7, 'media':8, 'sport':9}

y_train = list(map(topics.get, y_train))
y_test = list(map(topics.get, y_test))

# настройка и обучение классификатора
param_grid = [{'alpha': 0.0001 * np.arange(1,11)}]
multi_roc = make_scorer(roc_auc_score, average='weighted', multi_class='ovr', needs_proba=True)

clf = MultinomialNB()
search = GridSearchCV(clf, param_grid, cv=5, scoring=multi_roc)
search.fit(X_train, y_train)

print("Выполнено!")

C:\Users\bayut\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Выполнено!


In [30]:
print(f"Лучшие параметры: {search.best_params_}")
print(f"ROC-AUC: {roc_auc_score(y_test, search.best_estimator_.predict_proba(X_test), multi_class='ovr')}")

Лучшие параметры: {'alpha': 0.0001}
ROC-AUC: 0.9667054965725029


## 4) Протестируем улучшенный метод с использованием TF-IDF и w2v

In [31]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: normalize_text(tokenize_text(x)))
X_tfidf = vectorizer.fit_transform(X)

#eменьшаем размерность
svd = TruncatedSVD(n_components=WORDVEC_LEN, random_state=SEED)
X_dense = svd.fit_transform(X_tfidf.T)

# cоздаем словарь эмбеддингов
keys = vectorizer.get_feature_names_out()
emb_dict = {keys[i]: X_dense[i] for i in range(len(keys))}

def get_doc_emb_with_tfidf(doc):
    res = np.zeros(WORDVEC_LEN)
    tokens = normalize_text(tokenize_text(doc))
    if not tokens:
        return res
    count = 0
    for word in tokens:
        if word in emb_dict:
            res += emb_dict[word]
            count += 1
        if word in w2v.wv.key_to_index:
            res += w2v.wv.get_vector(word)
            count += 1
    return res / max(count, 1)

# новые эмбеддинги
X_train_improved = [get_doc_emb_with_tfidf(doc) for doc in raw_train]
X_test_improved = [get_doc_emb_with_tfidf(doc) for doc in raw_test]

# нормализация
X_train_improved = scaler.fit_transform(X_train_improved)
X_test_improved = scaler.transform(X_test_improved)

# обучаем классификатор
search.fit(X_train_improved, y_train)
improved_score = roc_auc_score(y_test, search.best_estimator_.predict_proba(X_test_improved), multi_class='ovr')

print("Выполнено!")

C:\Users\bayut\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Выполнено!


In [32]:
print(f"ROC-AUC (улучшенный метод): {improved_score}")

ROC-AUC (улучшенный метод): 0.9643417560145897


## Можно сделать вывод, что алгоритм w2v уже эффективно учитывает контекст слов. Улучшенный метод в данном случае не привёл к значимому улучшению метрики качества